In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data_dir = "../input/aml-fab/"

In [ ]:
df = pd.read_csv(data_dir + "GSE147515_FAB.csv")
df.head()

In [ ]:
df = df.T
df.head()

In [ ]:
df.columns = df.iloc[1]
df.drop(["Unnamed: 0", "ID"], axis=0, inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
lbl = pd.read_csv(data_dir + "GSE147515_FAB_lbl.csv")
lbl.drop(["Unnamed: 0"], axis=1, inplace=True)
lbl.head()

In [ ]:
X = df.values
X.shape

In [ ]:
y = lbl.FAB.values
y.shape

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20 , random_state = 0, stratify=y)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier(
    loss="hinge",
    penalty="l1",
    n_jobs=-1,
    class_weight="balanced",
    learning_rate="optimal",
)
sgd_parameters = {
    "alpha": [10 ** x for x in range(-6, 0)]
}

gsSGD = GridSearchCV(sgd, sgd_parameters, cv=10, scoring="f1_micro", verbose=10, n_jobs=-1)

In [ ]:
gsSGD.fit(X_train, y_train)

In [ ]:
gsSGD.best_params_

In [ ]:
sgd_best = gsSGD.best_estimator_
gsSGD.best_score_

In [ ]:
sgd_best.coef_

In [ ]:
sgd_best.classes_

In [ ]:
top10 = []
for i in range(0, sgd_best.coef_.shape[0]):
    top10.append(np.argsort(sgd_best.coef_[i])[-10:])

In [ ]:
top10

In [ ]:
for i in range(0, sgd_best.coef_.shape[0]):
    print(np.count_nonzero(sgd_best.coef_[i]))

In [ ]:
lbl.head()

In [ ]:
lbl.loc[lbl.FAB != "CTRL", "FAB"] = "AML"

In [ ]:
factor = pd.factorize(lbl.FAB)
lbl.FAB = factor[0]
definitions = factor[1]
lbl.head()

In [ ]:
y = lbl.FAB.values
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20 , random_state = 0, stratify=y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(random_state=0)
lasso_parameters = {
    "alpha": [10 ** x for x in range(-6, 1)]
}

gsLASSO = GridSearchCV(lasso, lasso_parameters, cv=10, scoring="r2", verbose=10, n_jobs=-1)

In [ ]:
gsLASSO.fit(X_train, y_train)

In [ ]:
gsLASSO.best_params_ #0.001

In [ ]:
lasso_best = gsLASSO.best_estimator_
gsLASSO.best_score_

In [ ]:
lasso_best.coef_

In [ ]:
np.count_nonzero(lasso_best.coef_)

In [ ]:
np.argsort(lasso_best.coef_)[-10:]

In [ ]:
df.columns.shape

In [ ]:
df.columns[np.argsort(lasso_best.coef_)[-10:]]